In [5]:
import os
from langchain.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import Pinecone  # ✅ Correct import for Pinecone
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv()

# ✅ Hugging Face API Key (Ensure it's set in your environment variables)
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not HUGGINGFACE_API_KEY:
    raise ValueError("❌ Missing Hugging Face API Key. Set it as HUGGINGFACEHUB_API_TOKEN")

# ✅ Load DeepSeek-R1 via API Inference
llm = HuggingFaceHub(
    repo_id="deepseek-ai/DeepSeek-R1",  # Model name
    model_kwargs={"temperature": 0.5, "max_length": 2048},
    huggingfacehub_api_token=HUGGINGFACE_API_KEY
)

# ✅ Initialize Pinecone Client
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# ✅ Ensure the new Pinecone index name is set
index_name = os.getenv("PINECONE_INDEX_V2")  # ✅ Updated to use the latest index
if not index_name:
    raise ValueError("❌ PINECONE_INDEX_V2 is not set. Check your .env file.")

# ✅ Ensure the index exists before using it
existing_indexes = [idx["name"] for idx in pinecone_client.list_indexes()]
if index_name not in existing_indexes:
    raise ValueError(f"❌ Index '{index_name}' does not exist in Pinecone. Please create it first.")

# ✅ Initialize Pinecone Index
index = pinecone_client.Index(index_name)

# ✅ Initialize the same embedding model used for storing embeddings
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# ✅ Load the vector store for retrieval
vectorstore = PineconeVectorStore(
    index=index,  # ✅ Connects to the updated Pinecone index
    embedding=huggingface_embeddings,
    text_key="text"  # ✅ Specifies where text metadata is stored
)

# ✅ Define the specific call ID for testing
test_call_id = "call_002"  # Change this to test different calls

# ✅ Create a retriever from the vector store with metadata filtering
retriever = vectorstore.as_retriever(
    search_type="similarity",  
    search_kwargs={
        "k": 3,  # Retrieve top 3 most relevant chunks
        "filter": {"call_id": test_call_id}  # ✅ Filter results by specific call ID
    }
)


# ✅ Define the Prompt Template for RAG-based Q&A
prompt_template = """
You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: {context}
</ctx>
-------
<hs>
Chat History: {chat_history}
</hs>
-------
Question: {question}

Useful Answer:
"""


PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "chat_history", "question"]  # ✅ Includes chat history now
)


# ✅ Memory buffer to handle multiple interactions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question"
)

# ✅ Define Retrieval-Based Q&A Chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT, "memory": memory}
)

In [ ]:
memory.clear() # para resetear memoria (si no se hace, se puede superar el limite de tokens y da error)

In [6]:
query = "how much is the amount per month the client would need to pay?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That sounds better. What would the payments look like?
Agent: It would be about $30 per month for 12 months.
Client: Okay, but what happens if I miss a payment?

Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t ma

In [7]:
query = "what is the agent very sorry to hear?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t make a payment right now.
Agent: I’m very sorry to hear that. Would a one-month extension help?
Client: It would, but I don’t know if I’ll have a job by then.

Client: Okay, but what happens if I miss a payment?
Agent: If you notify us ahead of time, we can reschedule without late fees.
Client: That’s really helpful. Will th

In [8]:
query = "make a summary of the call"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That sounds better. What would the payments look like?
Agent: It would be about $30 per month for 12 months.
Client: Okay, but what happens if I miss a payment?

Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t ma

In [9]:
query = "what is this call about?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That sounds better. What would the payments look like?
Agent: It would be about $30 per month for 12 months.
Client: Okay, but what happens if I miss a payment?

Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t ma

In [10]:
query = "what will John send to the client?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That sounds better. What would the payments look like?
Agent: It would be about $30 per month for 12 months.
Client: Okay, but what happens if I miss a payment?

Client: Okay, but what happens if I miss a payment?
Agent: If you notify us ahead of time, we can reschedule without late fees.
Client: That’s 

In [12]:
query = "can you make a summary of the questions I've asked about this call?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t make a payment right now.
Agent: I’m very sorry to hear that. Would a one-month extension help?
Client: It would, but I don’t know if I’ll have a job by then.

Client: It would, but I don’t know if I’ll have a job by then.
Agent: We also offer hardship plans that extend payments over 12 months with no penalties.
Client: That